In [1]:

%pprint

Pretty printing has been turned OFF



----
# Debug get_feature_dict_list

In [1]:

%%time
%run ../py/sql_utlis.py

su = SqlUtilities()
conn, cursor = su.get_jh_conn_cursor()

Wall time: 3.3 s


In [3]:

# Get the files in the child strings list
sql_str = '''
    SELECT
        fn.[file_name_id],
        RTRIM(fn.[file_name]) AS file_name
    FROM [Jobhunting].[dbo].[FileNames] fn'''
filenames_df = pd.DataFrame(su.get_execution_results(cursor, sql_str, verbose=False))
CHILD_STRS_LIST_DICT = su.s.load_object('CHILD_STRS_LIST_DICT')
mask_series = filenames_df.file_name.isin(list(CHILD_STRS_LIST_DICT.keys()))

In [6]:

row_series = filenames_df[mask_series].sample(1).squeeze()
file_name_id = row_series.file_name_id
file_name = row_series.file_name.strip()

In [ ]:

# Get the child strings list for the file
sql_str = '''
    SELECT np.[navigable_parent]
    FROM
        [Jobhunting].[dbo].[NavigableParentSequence] nps INNER JOIN
        [Jobhunting].[dbo].[NavigableParents] np ON
        nps.[navigable_parent_id] = np.[navigable_parent_id]
    WHERE nps.[file_name_id] = {}
    ORDER BY nps.[sequence_order] ASC'''.format(file_name_id)
child_strs_list = pd.DataFrame(su.get_execution_results(cursor, sql_str,
                                                        verbose=False)).navigable_parent.tolist()
child_strs_list

In [ ]:

# Get the child tags list for the file
sql_str = '''
    SELECT ht.[header_tag]
    FROM
        [Jobhunting].[dbo].[HeaderTagSequence] hts INNER JOIN
        [Jobhunting].[dbo].[HeaderTags] ht ON
        hts.[header_tag_id] = ht.[header_tag_id]
    WHERE hts.[file_name_id] = {}
    ORDER BY hts.[sequence_order] ASC'''.format(file_name_id)
child_tags_list = [x.strip() for x in pd.DataFrame(su.get_execution_results(cursor,
                                                                            sql_str)).header_tag.tolist()]
child_tags_list

In [ ]:

# Assume the is_header feature for each item in the sequence is not None
su.get_feature_dict_list(cursor, child_tags_list, child_strs_list)

In [ ]:

get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)